In [3]:
import math
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import numpy as np
import csv

In [4]:
class TrackPoint:
    
    def __init__(self, time, lat, lon):
        self.time = time
        self.lat = lat # latitude
        self.lon = lon # longitude


class Track:
    
    def __init__(self):
        self.points = [] # track points

    def gps_distance(self, p1, p2):
        # Haversine Formula
        lat1, lon1 = p1.lat, p1.lon
        lat2, lon2 = p2.lat, p2.lon
        Earth_radius = 6371  # km
        dlat = math.radians(lat2 - lat1)
        dlon = math.radians(lon2 - lon1)
        a = (math.sin(0.5 * dlat) * math.sin(0.5 * dlat) +
             math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
             math.sin(0.5 * dlon) * math.sin(0.5 * dlon))
        return 2 * Earth_radius * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    def near_point(self, point, radius):
        for p in self.points:
            if self.gps_distance(p, point) < radius:
                return True
        return False
    
    @property
    def size(self):
        return len(self.points)
    
    @property
    def is_epmty(self):
        return self.size == 0
    
    @property
    def has_bad_timing(self):
        dt, dd, dv = self.segments
        bad_timing = False
        for t in dt:
            if t > 5 * 60:
                # GPS tracker didn't respond for more than 5 minutes
                bad_timing = True
                break
        return bad_timing
    
    @property
    def has_small_distance(self):
        # total distance is less than 100 m
        return self.travel_distance < 0.1
    
    @property
    def origin(self):
        return None if self.is_epmty else self.points[0]
    
    @property
    def destination(self):
        return None if self.is_epmty else self.points[-1]
    
    @property
    def idle_time(self):
        t = 0.0
        if not self.is_epmty:
            ts, ds, _ = self.segments
            n = len(ts)
            if n > 0:
                for i in range(n):
                    if ds[i] == 0:
                        t += ts[i]
        return t
    
    @property
    def travel_time(self):
        return 0 if self.is_epmty else (self.destination.time - self.origin.time).seconds
    
    @property
    def travel_distance(self):
        td = 0.0
        if not self.is_epmty:
            _, dd, _ = self.segments
            td = sum(dd)
        return td
    
    @property
    def mean_speed(self):
        ms = 0.0
        if not self.is_epmty:
            _, _, vs = self.segments
            if len(vs) != 0:
                ms = 1.0 * sum(vs) / len(vs)
        return ms
    
    @property
    def aver_speed(self):
        return 0 if self.travel_time == 0 else 3600 * self.travel_distance / self.travel_time
    
    @property
    def irr_coef(self):
        return 1.0 * self.travel_distance / self.gps_distance(self.origin, self.destination)
    
    @property
    def segments(self):
        dt, dist, velocity = [], [], []
        if not self.is_epmty:
            for i in range(self.size - 1):
                dt.append((self.points[i + 1].time - self.points[i].time).seconds)
                dist.append(self.gps_distance(self.points[i], self.points[i + 1]))
                velocity.append(3600.0 * dist[-1] / dt[-1])
        return dt, dist, velocity
    

class GPX:
    
    def __init__(self, file_name):
        tree = ET.parse(file_name)
        self.tracks = self.get_tracks(tree)    
    
    def get_tracks(self, tree):
        trks = []
        root = tree.getroot()
        for trk in root:
            # print trk.tag, trk.attrib
            for trkseg in trk:
                tr = Track()
                for trkpt in trkseg:
                    tstr = trkpt[0].text # 2017-05-31T22:03:41Z
                    year, month, day = tstr[:4], tstr[5:7], tstr[8:10]
                    hour, minute, sec = tstr[11:13], tstr[14:16], tstr[17:19]
                    time = datetime(int(year), int(month), int(day), 
                                    int(hour), int(minute), int(sec))
                    if tr.is_epmty or time != tr.points[-1].time: # exclude repetitions
                        tr.points.append(TrackPoint(time=time,
                                                    lat=float(trkpt.attrib.get('lat')),
                                                    lon=float(trkpt.attrib.get('lon'))))
                # print "Points in track:", len(tr.points)
                trks.append(tr)
        return trks

In [5]:
class Trip:
    
    def __init__(self, trip_id, duration, distance, idle, irr_coef):
        self.id = trip_id
        self.duration = duration
        self.distance = distance
        self.idle = idle
        self.irr_coef = irr_coef
        
    @property
    def aver_velocity(self):
        return 0.0 if self.duration == 0 else 3600.0 * self.distance / self.duration
    
    @property
    def pure_velocity(self):
        return 0.0 if self.duration == 0 or self.duration == self.idle \
                   else 3600.0 * self.distance / (self.duration - self.idle)

In [17]:
gpx = GPX('gpx-data/wavelo_clean_10001_20000.gpx')

In [18]:
f = open('gpx-data/trips_clean_10001_20000.txt', 'w')
idx = 10000

for track in gpx.tracks:
    idx += 1
    f.write(str(idx) + '\t' + str(track.travel_time) + '\t' + 
            str(track.travel_distance) + '\t' + str(track.idle_time) + '\t' +
            str(track.irr_coef) + '\n')
f.close()